In [1]:
import sys
sys.path.append("../../")
from data.Frame import Frame
from bk import show_weighted_average, show_group_total, get_pandas_dataframe, line_chart, scatter_plot
from bokeh.io import output_notebook
from bokeh.plotting import figure, output_file, show, reset_output
from sources.unpd.indicators import get_projection_by_groups as gpp
from sources.cdp.indicators import get_indicators as get_indicators_cdp_2018
from sources.worldbank.datatable import get_data_table as get_wb_tb
reset_output()
output_notebook()

Loading BokehJS ...

# Human and Social development in the LDCs

# The Least Developed Countries

+ The category of the least developed countries was created in 1971

+ Least developed countries (LDCs) are low-income countries confronting severe structural impediments to sustainable development. They are highly vulnerable to economic and environmental shocks and have low levels of human assets.


Least developed countries (LDCs) are **low-income countries** confronting severe **structural** impediments to sustainable development. They are highly **vulnerable to economic and environmental shocks** and have **low levels of human assets**.

## The criteria to dentify LDCs
The Committee for Development Policy has established three criteria for including and graduating countries from the category:
    
### The Economic Vulnerability Index (EVI)
    
### The Human Assets Index (HAI)

### GNI per capita

In [2]:
cdp_data_2018 = get_indicators_cdp_2018(['LDCs'])
entities = cdp_data_2018.get_column('entity').get_as_array()
evi = cdp_data_2018.get_column('evi').get_as_array()
hai = cdp_data_2018.get_column('hai').get_as_array()
gni = cdp_data_2018.get_column('gni_per_capita_dollars').get_as_array()
gni_normalized = [(x / sum(gni))*30 for x in gni]

In [3]:
p = scatter_plot(evi, hai, gni_normalized, entities)
show(p)

# The Istanbul Programme of Action

# Objectives

27. The overarching goal of the Programme of Action for the decade 2011-2020 is to overcome the structural challenges faced by the least developed countries in order to eradicate poverty, achieve internationally agreed development goals and enable graduation from the least developed country category.



28. Guided by the overarching goal, national policies of least developed countries and international support measures during the decade will focus on the following specific objectives with the aim of enabling half the number of least developed countries to meet the criteria for graduation by 2020:

**(a)** Achieve sustained, equitable and inclusive economic growth in least developed countries, to at least the level of 7 per cent per annum, by strengthening their productive capacity in all sectors through structural transformation and overcoming their marginalization through their effective integration into the global economy, including through regional integration;

**(b) Build human capacities by fostering sustained, equitable and inclusive human and social development, gender equality and the empowerment of women;**

**(c)** Reduce the **vulnerability** of least developed countries to economic, natural and environmental shocks and disasters, as well as climate change, and enhance their ability to meet these and other challenges through strengthening their resilience;

**(d)** Ensure enhanced financial resources and their effective use for least developed countries’ development, including through domestic resource mobilization, ODA, external debt relief, foreign direct investment and remittances;

**(e)** Enhance good governance at all levels, by strengthening democratic processes, institutions and the rule of law; increasing efficiency, coherence, transparency and participation; protecting and promoting human rights; and reducing corruption, and strengthen least developed country Governments’ capacity to play an effective role in their economic and social development.

# IV. Priority areas for action

43. The actions will be organized by priority areas as:

A. Productive capacity

B. Agriculture, food security and rural development

C. Trade

D. Commodities

**E. Human and social development**

F. Multiple crises and other emerging challenges ▪Economic shocks

H. Good governance at all levels

# E. Human and social development

+ Education and training

+ Population and primary health

+ Youth development

+ Shelter

+ Water and sanitation

+ Gender equality and empowerment of women 

+ Social protection

# Population and primary health

In [4]:
years = list(map(str, list(range(1960,2018))))
result = show_group_total(years, 'SP.POP.TOTL', ['LDCs'])

In [5]:
show(line_chart(gpp(['LDCs', 'Developing excluding LDCs', 'OECD']), "Projected population, thousands", 
                x_axis_label='Years', y_axis_label='Projected population, thousands', height=400, width=1000))

In [6]:
import sources
data = sources.unpd.indicators.get_projection_by_group('LDCs')

In [7]:
# Rural population as a percentage of total population
result = show_weighted_average(years, 'SP.RUR.TOTL.ZS', 'SP.POP.TOTL', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD'])

In [8]:
result = show_group_total(years, 'SP.RUR.TOTL', ['LDCs'])

In [9]:
years = list(map(str, list(range(2000,2018))))
# Population 0-14
result = show_weighted_average(years, 'SP.POP.0014.TO.ZS', 'SP.POP.TOTL', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD', 'UMICs'])

In [10]:
show(line_chart(gpp(['LDCs', 'Developing excluding LDCs', 'OECD', 'UMICs']), 
                "Projected population, thousands", x_axis_label='Years', 
                y_axis_label='Projected population, thousands', height=400, width=900))

In [11]:
# Prevalence of undernourishment (% of population)
years = list(map(str, list(range(2000,2018))))
result = show_weighted_average(years, 'SN.ITK.DEFC.ZS', 'SP.POP.TOTL', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD'])

In [12]:
# Food imports as a percentage of merchandise imports
years = list(map(str, list(range(2000,2018))))
result = show_weighted_average(years, 'TM.VAL.FOOD.ZS.UN', 'TM.VAL.MRCH.CD.WT', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD'])

In [13]:
# Vulnerable employment
years = list(map(str, list(range(2000,2018))))
result = show_weighted_average(years, 'SL.EMP.VULN.ZS', 'SP.POP.TOTL', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD'])

In [14]:
# Female vulnerable employment
years = list(map(str, list(range(2000,2018))))
result = show_weighted_average(years, 'SL.EMP.VULN.FE.ZS', 'SL.TLF.TOTL.IN', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD'])

In [15]:
# Male vulnerable employment
years = list(map(str, list(range(2000,2018))))
result = show_weighted_average(years, 'SL.EMP.VULN.MA.ZS', 'SL.TLF.TOTL.IN', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD'])

In [16]:
# Life expectancy
years = list(map(str, list(range(2000,2018))))
result = show_weighted_average(years, 'SP.DYN.LE00.IN', 'SP.POP.TOTL', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD'])

In [17]:
# Female life expectancy
years = list(map(str, list(range(2000,2018))))
result = show_weighted_average(years, 'SP.DYN.LE00.FE.IN', 'SP.RUR.TOTL', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD'])

In [18]:
# Maternal mortality ratio
years = list(map(str, list(range(2000,2018))))
result = show_weighted_average(years, 'SH.STA.MMRT', 'SP.POP.TOTL.FE.IN', ['LDCs', 'Developing excluding LDCs'])

In [19]:
# Infant mortality rate
years = list(map(str, list(range(2000,2018))))
result = show_weighted_average(years, 'SP.DYN.IMRT.IN', 'SP.POP.TOTL', ['LDCs', 'Developing excluding LDCs'])

# Education and training

In [20]:
# School enrollment, primary
years = ['2011', '2016']
result = show_weighted_average(years, 'SE.PRM.ENRR', 'SP.POP.TOTL', 
                               ['LDCs'])

In [21]:
# School enrollment, secondary
years = list(map(str, list(range(2010,2018))))
result = show_weighted_average(years, 'SE.SEC.ENRR', 'SP.POP.TOTL', 
                               ['LDCs', 'Developing excluding LDCs', 'UMICs'])

In [22]:
get_wb_tb(years, 'SE.SEC.ENRR', [['LDCs']]).as_pandas()

,entity,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0
0,Afghanistan,50.56725,51.75691,53.61805,53.57513,52.58290,52.59281,50.64540,53.79764
1,Angola,26.25922,26.48254,NaN,NaN,NaN,NaN,50.67148,NaN
2,Bangladesh,51.63770,52.43137,55.20997,60.16874,NaN,65.62919,71.41584,69.66317
3,Benin,NaN,49.12759,NaN,55.96562,56.23654,58.84676,59.04252,NaN
4,Bhutan,63.56648,67.24879,71.69738,75.61265,82.71892,84.01727,85.34537,87.86760
5,Burkina Faso,21.90269,23.83593,25.93309,28.36596,30.35403,33.71873,35.78715,38.11725
6,Burundi,23.29063,26.12250,28.71599,33.41147,39.41126,44.34693,47.47817,49.25443
7,Cambodia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Central African Republic,NaN,17.62453,17.39388,NaN,NaN,NaN,15.03944,17.13538
9,Chad,22.58857,23.06594,22.32109,24.67565,23.03966,22.41506,22.55660,NaN


In [23]:
# School enrollment, tertiary
years = list(map(str, list(range(2010,2015))))
result = show_weighted_average(years, 'SE.TER.ENRR', 'SP.POP.TOTL', 
                               ['LDCs', 'Developing excluding LDCs', 'UMICs'])

In [24]:
# Pupil teacher ratio
years = list(map(str, list(range(2000,2018))))
result = show_weighted_average(years, 'SE.PRM.ENRL.TC.ZS', 'SP.POP.TOTL', 
                               ['LDCs', 'Developing excluding LDCs'])

In [25]:
# Trained teachers in primary
years = list(map(str, list(range(2005,2015))))
result = show_weighted_average(years, 'SE.PRM.TCAQ.ZS', 'SE.PRM.TCHR', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD'])

In [26]:
# Trained teachers secondary
years = list(map(str, list(range(2005,2015))))
result = show_weighted_average(years, 'SE.SEC.TCAQ.ZS', 'SE.SEC.TCHR', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD'])

# Youth development

In [27]:
# Literacy rate, youth male (% of male ages 15-24)
result = show_weighted_average(years, 'SE.ADT.1524.LT.MA.ZS', 'SP.POP.TOTL', 
                               ['LDCs', 'Developing excluding LDCs'])

In [28]:
# Literacy rate, youth female (% of females ages 15-24)
years = list(map(str, list(range(2011,2018))))
result = show_weighted_average(years, 'SE.ADT.1524.LT.FE.ZS', 'SP.POP.TOTL', 
                               ['LDCs', 'Developing excluding LDCs', 'UMICs'])

# Shelter

In [29]:
# Percentage of urban population living in slums
years = ['2000', '2005', '2007', '2009', '2014']
result = show_weighted_average(years, 'EN.POP.SLUM.UR.ZS', 'SP.URB.TOTL', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD'])

In [30]:
years = list(map(str, list(range(1960,2018))))
# Urban population as a percentage of total population
result = show_weighted_average(years, 'SP.URB.TOTL.IN.ZS', 'SP.POP.TOTL', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD'])

In [31]:
result = show_group_total(years, 'SP.URB.TOTL', ['LDCs'])

In [32]:
# Growth of urban population
years = list(map(str, list(range(2000,2018))))
result = show_weighted_average(years, 'SP.URB.GROW', 'SP.URB.TOTL', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD']) 

# Water and sanitation

In [33]:
# Access to safely managed sanitation services
years = list(map(str, list(range(2000,2018))))
result = show_weighted_average(years, 'SH.STA.SMSS.ZS', 'SP.POP.TOTL', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD']) 

In [34]:
# Basic water access
years = list(map(str, list(range(2000,2018))))
result = show_weighted_average(years, 'SH.H2O.BASW.ZS', 'SP.POP.TOTL', ['LDCs', 'Developing excluding LDCs'])

In [35]:
# Basic water access rural
years = list(map(str, list(range(2000,2018))))
result = show_weighted_average(years, 'SH.H2O.BASW.RU.ZS', 'SP.RUR.TOTL', ['LDCs', 'Developing excluding LDCs'])

# Gender equality and the empowerment of women

In [36]:
# Basic water access rural
years = ['2004', '2007', '2009', '2014', '2016']
result = show_weighted_average(years, 'SH.FPL.SATM.ZS', 'SP.POP.TOTL.FE.IN', 
                               ['LDCs', 'Developing excluding LDCs'])

In [37]:
# Basic water access rural
years = ['2000', '2003', '2006', '2009', '2014', '2017']
result = show_weighted_average(years, 'SP.ADO.TFRT', 'SP.POP.TOTL.FE.IN', 
                               ['LDCs', 'Developing excluding LDCs'])

# Social protection

In [38]:
# Age dependency ratio (% of working-age population)
years = list(map(str, list(range(2000,2018))))
result = show_weighted_average(years, 'SP.POP.DPND', 'SP.POP.1564.TO', ['LDCs', 'Developing excluding LDCs', 'OECD'])

In [39]:
#Employment to population ratio, 15+, total (%) (modeled ILO estimate)
# Age dependency ratio (% of working-age population)
years = list(map(str, list(range(2000,2018))))
result = show_weighted_average(years, 'SL.EMP.TOTL.SP.ZS', 'SP.POP.TOTL', ['LDCs', 'Developing excluding LDCs', 'OECD'])

# Some very general ideas

# The world is becoming increasingly unequal

# Great progress in certain areas, still an awful lot to do

# Sustainable development in the LDCs won't happen without addressing gender inequalities

# The SDGs and other internationally agreed development goals won't happen without accelerated progress in the LDCs.

# Many thanks

Let's stay in touch

Tomás González
gonzalezt@un.org
